# Color Filter

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt # für RaspberryPi Code entfernen

img = cv2.imread('Bild3.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

lower_red = np.array([0, 100, 110])
upper_red = np.array([345, 255, 255])

lower_blue = np.array([90, 95, 50])
upper_blue = np.array([130, 255, 255])

lower_yellow = np.array([25, 100, 100])
upper_yellow = np.array([30, 255, 255])

red_mask = cv2.inRange(hsv, lower_red, upper_red)
blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

final_mask = red_mask + blue_mask + yellow_mask
filter = cv2.bitwise_and(img, img, mask = final_mask)

bilateral = cv2.bilateralFilter(filter, 15,100,100) #anpassen
plt.imshow(bilateral) # für RaspberryPi Code entfernen
plt.show() # für RaspberryPi Code entfernen

result = cv2.cvtColor(filter, cv2.COLOR_BGR2RGB)
bilateral = cv2.bilateralFilter(result, 15,100,100) #anpassen
cv2.imwrite("Cubes3.png", bilateral)

# Depth Map

In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt # für RaspberryPi Code entfernen
from matplotlib.colors import LinearSegmentedColormap

filename = ("Cubes2.png")

model_type = "DPT_Large"
midas = torch.hub.load("intel-isl/MiDas", model_type)

midas_transforms = torch.hub.load("intel-isl/MiDas", "transforms")
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform

device = torch.device("cpu")
img = cv2.imread(filename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

input_batch = transform(img).to(device)


with torch.no_grad():
    prediction = midas(input_batch)

    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False,
    ).squeeze()
    
output = prediction.cpu().numpy()

plt.imshow(output, cmap='gray') # für RaspberryPi Code entfernen
plt.show() # für RaspberryPi Code entfernen

cv2.imwrite("DepthMap2.png", output)

# Image Segmentation

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt # für RaspberryPi Code entfernen

depth_map = cv2.imread("DepthMap2.png", cv2.IMREAD_GRAYSCALE)
img = cv2.imread("Cubes2.png")

threshold_distance = 14

mask = depth_map < threshold_distance

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img_rgb[mask] = [0,0,0]

gray_image = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)


threshold = 10
mask_black = cv2.threshold(gray_image, threshold, 255, cv2.THRESH_BINARY)[1]
mask_white = cv2.bitwise_not(mask_black)

img[mask_white == 255] = [255, 255, 255]
cv2.imwrite("Result2.png", img)

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img) # für RaspberryPi Code entfernen
plt.show() # für RaspberryPi Code entfernen

# Test

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
 
imgL = cv.imread('Cubes2.png', cv.IMREAD_GRAYSCALE)
imgR = cv.imread('Cubes2.png', cv.IMREAD_GRAYSCALE)
 
stereo = cv.StereoBM.create(numDisparities=16, blockSize=15)
disparity = stereo.compute(imgL,imgR)
plt.imshow(disparity,'gray')
plt.show()

In [1]:
import wget

In [3]:
!git clone https://github.com/compphoto/BoostingMonocularDepth.git

fatal: destination path 'BoostingMonocularDepth' already exists and is not an empty directory.


In [4]:
wget.download("https://sfu.ca/~yagiz/CVPR21/latest_net_G.pth")

100% [......................................................................] 318268048 / 318268048

'latest_net_G.pth'

In [ ]:
%cd BoostingMonocularDepth/pix2pix/

In [ ]:
!mkdir -p checkpoints

In [ ]:
%cd /BoostingMonocularDepth/pix2pix/checkpoints/

In [ ]:
!mkdir -p mergemodel

In [2]:
%cd C:\Users\Silvan Mangold\Desktop\Studium\Github\PREN\TA.BA_PREN_PY_ObjectDetection\Tiefenfilter

C:\Users\Silvan Mangold\Desktop\Studium\Github\PREN\TA.BA_PREN_PY_ObjectDetection\Tiefenfilter


C:\Users\Silvan Mangold\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
!move latest_net_G.pth /BoostingMonocularDepth/pix2pix/checkpoints/mergemodel/

In [ ]:
wget.download("https://github.com/AlexeyAB/MiDaS/releases/download/midas_dpt/midas_v21-f6b98070.pt")
!move midas_v21-f6b98070.pt BoostingMonocularDepth/midas/model.pt

In [ ]:
wget.download("https://huggingface.co/lllyasviel/Annotators/resolve/850be791e8f704b2fa2e55ec9cc33a6ae3e28832/res101.pth")
!move res101.pth BoostingMonocularDepth/res101.pth

In [ ]:
%cd BoostingMonocularDepth/

In [ ]:
!python BoostingMonocularDepth/run.py --Final --data_dir inputs/ --output_dir outputs_leres/ --depthNet 0